In [1]:
# %% [markdown]
# Affichage des 3 premières décimales de pi

# %%
from math import pi
print(f"{pi:.3f}")

# %% [markdown]
# Affichage des 10 premières décimales de pi

# %%
print(f"{pi:.10f}")

# %% [markdown]
# Affichage des 15 premières décimales de pi

# %%
print(f"{pi:.15f}")

# %% [markdown]
# Affichage des 50 premières décimales de pi

# %%
print(f"{pi:.50f}")

3.142
3.1415926536
3.141592653589793
3.14159265358979311599796346854418516159057617187500
